In [1]:
import pandas as pd 
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains
import time
import sys
import os
import math
import numpy
import xlwt
import urllib.request
import requests

keyword  = input("크롤링할 키워드를 입력해주세요.:")
how_many_videos = int(input("크롤링할 영상의 수를 입력해주세요:"))
how_many_comments = int(input("크롤링할 댓글의 수를 입력해주세요:"))
f_dir = input("결과 파일을 저장할 폴더명을 작성하세요.(예:c:\\data\\):")
n = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (n.tm_year, n.tm_mon, n.tm_mday, n.tm_hour, n.tm_min, n.tm_sec)
os.makedirs(f_dir+s+'-'+keyword)
os.chdir(f_dir+s+'-'+keyword)
f_txt=f_dir+s+'-'+keyword+'\\'+s+'-'+keyword+'.txt'
f_csv=f_dir+s+'-'+keyword+'\\'+s+'-'+keyword+'.csv'
f_xls=f_dir+s+'-'+keyword+'\\'+s+'-'+keyword+'.xlsx'
path = "C:\Temp\chromedriver_win32\chromedriver.exe"
driver = webdriver.Chrome(path)
driver.get("http://youtube.com")
time.sleep(1)
search_bar = driver.find_element(By.NAME, "search_query")
search_bar.send_keys(keyword)
search_bar.submit()
time.sleep(1)
for _ in range(int(how_many_videos / 5)+1):
    body = driver.find_element(By.CSS_SELECTOR, 'body')
    body.send_keys(Keys.END)
    time.sleep(1)
driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)

vds = []
soup = BeautifulSoup(driver.page_source, 'html.parser')
vd_rends = soup.find_all("ytd-video-renderer", class_="style-scope ytd-item-section-renderer")
for vd_r in vd_rends:
    v = vd_r.find("ytd-thumbnail", class_="style-scope ytd-video-renderer").find("a", class_="yt-simple-endpoint inline-block style-scope ytd-thumbnail").get("href")
    vds.append(v)
v_cnt = 0
nicks = []
cmts = []
dates = []
urls = []
v_nl = []
c_nl = []

for vd in vds:
    if (vd.startswith("/shorts")):
        continue
    driver.get("http://youtube.com" + vd)
    time.sleep(1)
    v_cnt += 1
    c_cnt = 0
    scroll_cnt = 0
    while len(cmts) < how_many_comments * how_many_videos:
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        cmt_elements = soup.find_all("ytd-comment-thread-renderer")
        if (scroll_cnt == 3) & (len(cmt_elements) < how_many_comments):
            v_cnt -= 1
            break
        for cmt_box in cmt_elements:
            if c_cnt >= how_many_comments:
                break
            if len(cmt_elements) >= how_many_comments:
                cmt_detail = cmt_box.find('ytd-comment-renderer').find('div', id='body').find('div', id='main')
                nicks.append(cmt_detail.find('div', id='header').find('div', id='header-author').find('h3').get_text().strip())
                dates.append(cmt_detail.find('div', id='header').find('div', id='header-author').find('yt-formatted-string', class_='published-time-text style-scope ytd-comment-renderer').get_text().strip())
                cmts.append(cmt_detail.find('div', id='comment-content').get_text().strip().replace("간략히", "").replace("자세히 보기", ""))
                urls.append("http://youtube.com" + vd)
                v_nl.append(v_cnt)
                c_nl.append(c_cnt)
                c_cnt += 1
        if c_cnt >= how_many_comments:
            break
        driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)
        scroll_cnt += 1
    if (v_cnt-1 == how_many_videos):
        break
        
df = pd.DataFrame()
df['영상번호'] = v_nl
df['댓글번호'] = c_nl
df['작성자'] = nicks
df['작성일자'] = dates
df['내용'] = cmts
df['url'] = urls
df.to_excel(f_xls)
df.to_csv(f_csv, encoding="utf-8-sig")
f = open(f_txt, 'a',encoding='UTF-8')
for i in range(len(v_nl)):
    f.write(str(v_nl[i]) + '번째 영상의 ' + str(c_nl[i]+1) + '번째 댓글입니다. ~~\n')
    f.write('1.작성자: ' + nicks[i] + '\n')
    f.write('2.작성일: ' + dates[i] + '\n')
    f.write('3.댓글내용: ' + cmts[i] + '\n')
    f.write('4.영상url: ' + urls[i] + '\n\n\n')
f.close()
print("요청하신 데이터 추출이 성공적으로 끝났습니다.")

크롤링할 키워드를 입력해주세요.:에비츄
크롤링할 영상의 수를 입력해주세요:20
크롤링할 댓글의 수를 입력해주세요:2
결과 파일을 저장할 폴더명을 작성하세요.(예:c:\data\):C:\Users\UIT801-\Desktop\20191469\3학년 1학기\SNS빅데이터\과제\SNS_기말과제\Q5\


C:\Users\UIT801-\AppData\Local\Temp\ipykernel_7440\36901644.py:29: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


요청하신 데이터 추출이 성공적으로 끝났습니다.
